In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sqlalchemy import create_engine

In [5]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('Labeled_data.txt', header=None, delim_whitespace=True, skiprows=37) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()
#shuffle data
labeled_data_sampled = labeled_data.sample(n=5, random_state=1)

In [21]:
engine = create_engine('sqlite:///save_LC.db', echo=False)
sqlite_connection = engine.connect()

circle_radius = 0.00028 # 1 arcsec = 0.00028 degress
t_format = "ipac_table"
table_format = "FORMAT=" + str(t_format)
flag_mask = 32768
mask = "BAD_CATFLAGS_MASK=" + str(flag_mask)
collect="COLLECTION="+"ztf_dr2"
numobs = "NOBS_MIN=20"
filter_band = "g"
label = []
SourceID =[]
start_time = time.time()
k = 0
for i in labeled_data_sampled.index:
    label.append(labeled_data_sampled.Type[i])
    SourceID.append(labeled_data_sampled.SourceID[i])
    ra = labeled_data_sampled.RAdeg[i]
    dec = labeled_data_sampled.DEdeg[i]
    circle = "POS=CIRCLE"+"+"+str(ra)+"+"+str(dec)+"+"+str(circle_radius)
    band = "BANDNAME="+ filter_band
    params = circle + "&" +  mask + "&" + numobs + "&" + collect + "&" + table_format
    
    url= "https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?" + params
    data = pd.read_csv(url, header=None, delim_whitespace=True, skiprows=55) # extract data
    header = pd.read_csv(url, header=None, sep='|', skiprows=50,usecols=range(1,25), nrows=1)
    data.columns = header.iloc[0].str.strip()
    lc = data
    sqlite_table = labeled_data_sampled.ID[i]
    lc.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
    print(k)
    k = k+1
end_time = time.time()# total time taken
print(f"Runtime of the program is {end_time} - {start_time}")

0
1
2
3
4
Runtime of the program is {end_time} - {start_time}


In [13]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781601 entries, 0 to 781600
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          781601 non-null  object 
 1   SourceID    781601 non-null  int64  
 2   RAdeg       781601 non-null  float64
 3   DEdeg       781601 non-null  float64
 4   Per         781601 non-null  float64
 5   R21         781601 non-null  float64
 6   phi21       781601 non-null  float64
 7   T0          781601 non-null  float64
 8   gmag        781601 non-null  float64
 9   rmag        781601 non-null  float64
 10  Per_g       781601 non-null  float64
 11  Per_r       781601 non-null  float64
 12  Num_g       781601 non-null  int64  
 13  Num_r       781601 non-null  int64  
 14  R21_g       781601 non-null  float64
 15  R21_r       781601 non-null  float64
 16  phi21_g     781601 non-null  float64
 17  phi21_r     781601 non-null  float64
 18  R2_g        781601 non-null  float64
 19  R2

In [28]:
print(labeled_data_sampled.ID[i])

ZTFJ185629.89-050650.2


In [26]:
df = pd.read_sql_query('SELECT * FROM "ZTFJ185629.89-050650.2" LIMIT 3',sqlite_connection)
df.head()

,index,oid,expid,hjd,mjd,mag,magerr,catflags,filtercode,ra,...,ccdid,qid,limitmag,magzp,magzprms,clrcoeff,clrcounc,exptime,airmass,programid
0,0,436103100068854,45152934,2.458206e+06,58205.529340,19.590504,0.088645,0,zg,284.124562,...,3,1,19.969999,26.302597,0.033256,-0.070608,0.000021,30,1.368,1
1,1,436103100068854,45346820,2.458208e+06,58207.468206,19.478760,0.083781,0,zg,284.124483,...,3,1,19.660000,26.260078,0.038592,-0.071844,0.000028,30,1.771,1
2,2,436103100068854,45352662,2.458208e+06,58207.526620,19.565712,0.087535,0,zg,284.124526,...,3,1,20.049999,26.347794,0.029796,-0.066110,0.000017,30,1.357,1


In [23]:
engine = create_engine('sqlite:///save_LC.db', echo=False)
sqlite_connection = engine.connect()

In [24]:
from sqlalchemy import MetaData
from sqlalchemy import Table

In [25]:
metadata = MetaData()
print(metadata.tables)

FacadeDict({})


In [28]:
LC_data = pd.read_csv('http://variables.cn:88/seldataz.php?SourceID=781604', header=None,skiprows=0) # extract data
header = pd.read_csv('http://variables.cn:88/seldataz.php?SourceID=781604', header=None,skiprows=0) # extract data
LC_data.columns = header.iloc[0].str.strip()

In [29]:
LC_data.head()

,SourceID,RAdeg,DEdeg,HJD,mag,e_mag,flag,band
0,SourceID,RAdeg,DEdeg,HJD,mag,e_mag,flag,band
1,781604,359.9977487,52.4258851,58270.46708,17.793,0.024,0,g
2,781604,359.9977487,52.4258851,58268.46436,17.796,0.024,0,g
3,781604,359.9977487,52.4258851,58262.47265,17.818,0.025,0,g
4,781604,359.9977487,52.4258851,58276.40193,17.796,0.024,0,g


In [27]:
header.head()

,0,1,2,3,4,5,6,7
0,SourceID,RAdeg,DEdeg,HJD,mag,e_mag,flag,band
1,781604,359.9977487,52.4258851,58270.46708,17.793,0.024,0,g
2,781604,359.9977487,52.4258851,58268.46436,17.796,0.024,0,g
3,781604,359.9977487,52.4258851,58262.47265,17.818,0.025,0,g
4,781604,359.9977487,52.4258851,58276.40193,17.796,0.024,0,g
